In [ ]:
!pip install transformers

In [ ]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

In [ ]:
!huggingface-cli login --token YOUR_TOKEN --add-to-git-credential

In [ ]:
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "Describe in great detail what you see in this image: "}
    ]}
]

In [ ]:
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

In [ ]:
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

In [ ]:
output = model.generate(**inputs, max_new_tokens=150)

In [ ]:
print(processor.decode(output[0]))